<a href="https://colab.research.google.com/github/ssktora/productions/blob/main/%E3%83%96%E3%83%A9%E3%83%83%E3%82%AF%E3%82%B8%E3%83%A3%E3%83%83%E3%82%AFBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# キー（チャネルシークレットトークン）とWEBHOOK（チャネルアクセストークン）を入力

In [ ]:
KEY = 'チャネルシークレットトークンを入力'
WEBHOOK = 'チャネルアクセストークンを入力'

# 提供された部分

In [ ]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

app = Flask(__name__)

line_bot_api = LineBotApi(KEY)
handler = WebhookHandler(WEBHOOK)

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

# 部品の関数

In [ ]:
import random

#カードを取る行為
def choice_card(tramp):
    card = random.choice(tramp)
    num =  tramp.index(card)
    tramp.pop(num)
    return card

#最終的な手札の合計値を調べる
def number_count(cards):
    result = 0
    ace_n = 0
    for i in range(len(cards)):
        if cards[i] == "A":
            ace_n += 1
        
        elif isinstance(cards[i], int):
            result += cards[i]
        
        else:
            result += 10
    
    if ace_n != 0:
        for i in range(ace_n):
            checker = result + 11
            if checker <= 21:
                result += 11
            else:
                result += 1
    
    return result

# 勝者を判定する関数
def judge(parent, child, parent_point, child_point, point):
    p_result = 21 - parent
    c_result = []
    for d in child:
        c_res = 21 - d
        c_result.append(c_res)
        
    if p_result < 0:
        parent_point -= int(point)
        append_point = (int(point) * number) / (number-1) 
        child_point = [x-int(point)+append_point for x in child_point]
        return parent_point, child_point
    
    c_best = 100
    for e in c_result:
        if e >= 0 and e < c_best:
            c_best = e
    best_index = []
    for i in range(len(c_result)):
        if c_result[i] == c_best:
            best_index.append(i)
    
    if p_result >= 0 and c_best == 100:
        parent_point += int(point)*(number-1)
        child_point = [x-point for x in child_point]
        return parent_point, child_point
    
    elif p_result > c_best:
        win_n = len(best_index)
        child_point = [x-point for x in child_point]
        parent_point -= point
        append_point = (point*number)/win_n
        for d in best_index:
            child_point[d] += append_point
            
        return parent_point, child_point
    
    elif p_result <= c_best:
        parent_point += point*(number-1)
        child_point = [x - point for x in child_point]
        return parent_point, child_point
    
#print(number_count([4, 6, "Q"]))
#print(judge(20, [19, 18], 1000, [1000, 1000], 100))



    

# Linebotの返答部分

In [ ]:
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global game_now
    global player_point
    global com_point
    global p_list
    global n
    global first
    global point
    global player_cards
    global com_cards
    global tramp
    global second
    global check
    global number
    global people
    global counter
    global nx
    global x2
    global z2
    input_text = event.message.text
    reply_text = []
    # 初期設定
    if input_text == "ブラックジャック":
        reply_text.append(TextSendMessage(text='ゲームを開始します。'))
        reply_text.append(TextSendMessage(text='何人でプレイしますか？'))
        game_now = 1
        people = True
    
    # 1ターン目①（現在の親と親が引いたカードを伝えと賭けポイントを聞く）
    elif people and game_now == 1:
        try:
            number = int(input_text)
            error_occur = False
        except:
            error_occur = True
        if not error_occur:
            counter = 0
            player_point = []
            player_cards = []
            p_list = []
            p = 1
            for i in range(number):
                player_point.append(1000)
                p_list.append(p)
                p += 1
                cards = []
                player_cards.append(cards)
            x2 = ""
            z2 = ""
            n = 0
            nx = n
            first = True
            check = True
            people = False
            tramp = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K",
                     "A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K",
                     "A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K",
                     "A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
            reply_text.append(TextSendMessage(text='現在の親は'+str(p_list[n])+'Pです。'))
            card = choice_card(tramp)
            player_cards[int(p_list[n])-1].append(card)
            reply_text.append(TextSendMessage(text=str(p_list[n])+"Pさんは" + str(player_cards[int(p_list[n])-1][0]) + "を引きました。"))
            reply_text.append(TextSendMessage(text=str(p_list[n])+'Pさんは何ポイント賭けますか？'))
    
    # 1ターン目②(親が賭けたポイントと子が1ターン目で引いたカードを伝え2ターン目に引くカードの枚数を聞く)
    elif first and game_now == 1 and check:
        try:
            point = int(input_text)
            error_occur = False
        except:
            error_occur = True
        if not error_occur:
            x = ""
            first = False
            reply_text.append(TextSendMessage(text=str(p_list[n])+"Pさんは" + str(point) + "ポイント賭けました。"))
            child_list = p_list.copy()
            child_list.remove(p_list[n])
            for i in range(len(child_list)):
                card = choice_card(tramp)
                player_cards[int(child_list[i])-1].append(card)
                x += str(child_list[i])+"Pさんは" + str(player_cards[int(child_list[i])-1][0]) + "を引きました。\n"
            print(player_cards)
            reply_text.append(TextSendMessage(text=x))
            reply_text.append(TextSendMessage(text=str(p_list[n])+'Pさんは何枚引きますか？'))
            second = True
    
    # 結果表示
    elif second and game_now == 1 and check:
        try:
            player_n = int(input_text)
            error_occur = False
        except:
            error_occur = True
        if not error_occur:
            for i in range(player_n):
                p_card = choice_card(tramp)
                print(player_cards)
                print(nx)
                print(p_list)
                player_cards[int(p_list[nx])-1].append(p_card)
                x2 += str(p_list[nx]) + "Pさんは" + str(p_card) + "を引きました。\n"
                if i == player_n - 1:
                    z2 += str(p_list[nx]) + "Pさんの手札は" + str(player_cards[int(p_list[nx])-1]) + "になりました。\n"
            counter += 1
            if nx == len(p_list) - 1:
                nx = 0
            else:
                nx += 1
            
            if counter < number:
                reply_text.append(TextSendMessage(text=str(p_list[nx]) + "Pさんは何枚引きますか"))
            else:
                second = False
                check = False
                c_result = []
                for i in range(number):
                    n_result = number_count(player_cards[i])
                    c_result.append(n_result)
                p_result = c_result[n]
                c_result.pop(n)
                reply_text.append(TextSendMessage(text=x2))
                reply_text.append(TextSendMessage(text=z2))
                parent_point = player_point.pop(n)
                child_point = player_point.copy()
                parent_point, child_point = judge(p_result, c_result, parent_point, child_point, point)
                player_point = child_point.copy()
                player_point.insert(n, parent_point)
                c_result.insert(n, p_result)
                x = ""
                y = ""
                for i in range(len(player_point)):
                    x += str(i+1)+"Pさん:"+str(c_result[i])+"\n"
                    y += "現在の"+str(i+1)+"Pさんのポイントは"+str(player_point[i])+"ポイントです。\n"
                reply_text.append(TextSendMessage(text=x))
                reply_text.append(TextSendMessage(text=y))
                reply_text.append(TextSendMessage(text="ゲームを継続しますか？継続する場合は「はい」終える場合は「いいえ」と入力してください。"))                
                
    # 「はい」と返答されたとき（現在の親と親が引いたカードを伝えと賭けポイントを聞く）           
    elif input_text == "はい" and game_now == 1:
        if n == len(p_list) - 1:
            n = 0
        else:
            n += 1
        nx = n
        x2 = ""
        z2 = ""
        counter = 0
        child_list = p_list.copy()
        child_list.remove(p_list[n])
        first = True
        check = True
        tramp = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K",
                 "A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K",
                 "A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K",
                 "A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
        player_cards = []
        for i in range(number):
            cards = []
            player_cards.append(cards)
        reply_text.append(TextSendMessage(text='現在の親は'+str(p_list[n])+'Pです。'))
        card = choice_card(tramp)
        player_cards[int(p_list[n])-1].append(card)
        reply_text.append(TextSendMessage(text=str(p_list[n])+"Pさんは" + str(player_cards[int(p_list[n])-1][0]) + "を引きました。"))
        reply_text.append(TextSendMessage(text=str(p_list[n])+'Pさんは何ポイント賭けますか？'))
　　# 「いいえ」と返答されたとき
    elif input_text == "いいえ" and game_now == 1:
        game_now = 0
        
        reply_text.append(TextSendMessage(text='ゲームを終了します。'))
            
    # メッセージが存在しない場合は「意味が分かりません。」と送信        
    if len(reply_text) == 0:
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text="意味が分かりません。"))
    # メッセージが存在する場合は送信
    else:
        line_bot_api.reply_message(event.reply_token,reply_text,)
        
        
        

# 実行

In [ ]:
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=19102)